In [2]:
"""

LINKS:
- https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group
- https://math.stackexchange.com/questions/832173/structure-of-g-invariant-bilinear-forms-over-finite-fields

NOTE: the theorem in MO question only guarantees a bilinear form, not a sesqilinear form.

""";

In [3]:
"""

From Dima Pasechnik:

Once you have U, you can compute the Cholesky decomposition (U = L\overline{L}^T, with L lower-triangular), or LDLT decomposition (U=LDL^T, with L_{ii}=1 for all i, L lower-triangular, and D a diagonal matrix with entries in the index 2 subfield) of it, which is a purely algebraic, exact, process, and works in any characteristic, as far as I can see.

Cholesky involves taking square roots in the index 2 subfield - but these are always available, as they exist is the quadratic extension (in the case of odd characteristic, or in the field itself in the case of characteristic 2). LDLT doesn't need this, but leaves you with a D for the matrix for the form, rather than I. However, D can still be factored by taking square roots, and it works by the same argument.

---

There is up to a linear, i.e. conjunction in  (?) isomorphism only one  for all  and primes . See e.g. the Altas of Finite groups by Conway et al.

This would imply the existence of a base change.

""";

In [4]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [5]:
def invariant_symmetric_sesquilinear_matrix(q,partition):
    """
    Computes the matrix of a S_n-invariant symmetric sesquilinear form.

    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 

    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.

    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.

    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    
    """

    # Define the group G and its rep'n as a Specht module, dimension
    n = sum(partition)
    SGA = SymmetricGroupAlgebra(GF(q^2), n)
    SM = SGA.specht_module(partition)
    G = SGA.group()
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(GF(q^2), 'u', d_rho^2)
    U_vars = R.gens()  # List of variable generators for U
    U = Matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):

        #compute \rho(g), transpose, conjugate
        rho_g = rho(Permutation(g))
        rho_g_T = rho_g.transpose()
        rho_g_conj = conjugate_pos_char(rho_g)
    
        # Compute lambda_g
        det_rho_g = det(rho_g)
        lambda_g = det_rho_g * (det_rho_g ** q)
    
        # Form the matrix equation \rho(g)^T*U*\overline{\rho(g)} = \lambda_g * U
        equation_matrix = rho_g_T*U*rho_g_conj - lambda_g * U
    
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
    
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
            
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
            
                # Append the row to the augmented system
                augmented_system.append(row)
    
        # Convert the augmented system to a matrix
        return Matrix(GF(q^2), augmented_system)

    #stack linear systems for each g in G
    total_system = matrix(GF(q^2),0,d_rho^2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mats = [matrix(GF(q^2),d_rho,d_rho,b) for b in null_space.basis()]

    #verify that a solution to the linear system satisfies the G-invariance property
    assert all(rho(g).transpose()*U_mats[0]*conjugate_pos_char(rho(g)) == U_mats[0] for g in G)
    
    return U_mats

In [6]:
#define a general bi/sesquilinear form with the matrix U
def sesquilinear_form(x,y,U):
    return x.transpose()*U*y

In [7]:
#ensure the resulting form is G-invariant, symmetric, bi/sesquilinear by symbolic verification
def check_form_properties(q,partition):
    #define the representation matrix corresponding to q, partition
    SGA = SymmetricGroupAlgebra(GF(q^2),sum(partition))
    SM = SGA.specht_module(partition)
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    G = SGA.group()

    #define variables as polynomial generators
    R_xy = PolynomialRing(GF(q^2), d_rho, var_array='x,y')
    x = matrix([R_xy.gens()[2*i] for i in range(d_rho)]).transpose()
    y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)]).transpose()
    R_xy_lambda = PolynomialRing(R_xy,'lambda')
    lambda_ = R_xy_lambda.gens()[0]

    #compute the bilinear form matrix. coerce over polynomial ring
    U_mats = invariant_symmetric_sesquilinear_matrix(q,partition)
    U_mat = U_mats[0]
    if len(U_mats) > 1:
        print("Space of G-invariant symmetric bilinear forms has dimension > 1 for la=",partition)
        print("Dimension of space=",len(U_mats))
    U_form = matrix(R_xy_lambda,U_mat)
    
    #check symmetric property
    symmetric = sesquilinear_form(x,y,U_form) == sesquilinear_form(y,x,U_form)
    
    #check G-invariance property
    G_invariant = all(sesquilinear_form(rho(g)*x,rho(g)*y,U_form) == sesquilinear_form(x,y,U_form) for g in G)
    
    #check sesquilinear property. ISSUE: lambda_^q is a power of the ring generator, i.e. doesn't simplify.
    first_arg = sesquilinear_form(lambda_*x,y,U_form) == lambda_*sesquilinear_form(x,y,U_form)
    second_arg = sesquilinear_form(x,lambda_*y,U_form) == lambda_*sesquilinear_form(x,y,U_form) #need to amend for conjugation
    bilinear = first_arg and second_arg

    return symmetric and G_invariant and bilinear

In [8]:
#print the checks for pairs (p,la) where p is a prime and la is a partition of n
for p in Primes()[:5]:
    print(f"p={p}")
    for n in range(3,6):
        print([(la,check_form_properties(p,la)) for la in Partitions(n)])
    print("---------")

p=2
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
Space of G-invariant symmetric bilinear forms has dimension > 1 for la= [3, 1, 1]
Dimension of space= 2
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=3
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=5
[([3], True), ([2, 1], True), ([1, 1, 1], True)]
[([4], True), ([3, 1], True), ([2, 2], True), ([2, 1, 1], True), ([1, 1, 1, 1], True)]
[([5], True), ([4, 1], True), ([3, 2], True), ([3, 1, 1], True), ([2, 2, 1], True), ([2, 1, 1, 1], True), ([1, 1, 1, 1, 1], True)]
---------
p=7
[([3], True), ([2, 1], True), ([1, 1, 1], True)]


In [9]:
#NOTE: for q=2, la=[3,1,1] we get a two dim'l space
invariant_symmetric_sesquilinear_matrix(2,[3,1,1])

[
[1 1 1 1 1 0]  [0 0 0 0 0 1]
[1 1 1 1 0 1]  [0 0 0 0 1 0]
[1 1 1 0 1 1]  [0 0 0 1 0 0]
[1 1 0 1 1 1]  [0 0 1 0 0 0]
[1 0 1 1 1 1]  [0 1 0 0 0 0]
[0 1 1 1 1 1], [1 0 0 0 0 0]
]

In [10]:
#the G-invariant, symmetric, bilinear form is not the standard form. likely a different basis
U_mat = invariant_symmetric_sesquilinear_matrix(5,[3,1])[0]; U_mat

[1 3 3]
[3 1 3]
[3 3 1]

In [11]:
#to explain q=2, la=[3,1,1] note that the one of the composition factors (the irreducible quotients from any composition series)
#has multiplicity 2. we see this by looking at the Brauer character of the composition factor, and see (1,1,1) appears twice.
S5 = SymmetricGroup(5)
SGA = S5.algebra(GF(2))
for la in Partitions(5):
    M = SGA.specht_module(la)
    print(la, [V.brauer_character() for V in M.composition_factors()])

[5] [(1, 1, 1)]
[4, 1] [(4, 1, -1)]
[3, 2] [(4, -2, -1), (1, 1, 1)]
[3, 1, 1] [(1, 1, 1), (4, -2, -1), (1, 1, 1)]
[2, 2, 1] [(1, 1, 1), (4, -2, -1)]
[2, 1, 1, 1] [(4, 1, -1)]
[1, 1, 1, 1, 1] [(1, 1, 1)]


In [12]:
#compute the Cholesky decomposition of the matrix U associated to the bilinear form
try:
    print("matrix is Hermition:",U_mat.is_hermitian())
    print(U_mat.cholesky())
except ValueError as e:
    print(e)

matrix is Hermition: True
[       1        0        0]
[       3 4*z2 + 3        0]
[       3 3*z2 + 1        2]


In [28]:
#print the matrix associated to the bilinear form for each (q,partition)
#note q needs to be large enough. p \nmid n! seems sufficient
for la in Partitions(4):
    U_mats = invariant_symmetric_sesquilinear_matrix(3,la)
    if len(U_mats) > 1:
        print("space of G-invariant symmetric bilinear forms has dimension greater than 1")
    print("la =",la)
    print("q =",3)
    print(U_mats[0])
    print("--------")

la = [4]
q = 3
[1]
--------
la = [3, 1]
q = 3
[1 2 2]
[2 1 2]
[2 2 1]
--------
la = [2, 2]
q = 3
[1 2]
[2 1]
--------
la = [2, 1, 1]
q = 3
[0 1 2]
[1 0 1]
[2 1 0]
--------
la = [1, 1, 1, 1]
q = 3
[1]
--------


In [19]:
"""
For a unitary change-of-basis, we must have change-of-basis matrix A s.t.:

(A\rho(g)A.inverse())^* A \rho(g)A.inverse() = A.inverse()^* \rho(g)^*A^*A\rho(g)A.inverse()

then we must have A^*A = U = LL^* ==> A = L^*

= (L^*).inverse()^* \rho(g)^* L^{**}L^* \rho(g) L^*.inverse() #substitution
= L.inverse() \rho(g)^T LL^* \rho(g) L^*.inverse() #assuming the rep'n \rho is defined over the base field F_q fixed by x |--> x^q
= L.inverse() \rho(g)^T U \rho(g) L^*.inverse() 
= L.inverse() U L^*.inverse() #by G-invariance
= Id

Thus \tilde(\rho}(g) = L^* \rho(g) L^*.inverse() should be a unitary matrix. 

We use the GAP `forms` package BaseChangeToCanonical method to find the matrix factorization U = LL^* over F_{q^2}.

""";

In [32]:
#define the representation matrix corresponding to q, partition
q = 17; la = [2,1,1]
SGA = SymmetricGroupAlgebra(GF(q^2),sum(la))
SM = SGA.specht_module(la)
rho = SM.representation_matrix
d_rho = SM.dimension()
G = SGA.group()

In [34]:
#NOTE: the built-in Cholesky decomposition is only returning U = LL^T, not LL^* with the x |--> x^q conjugation
#NOTE: we can create orthogonal representations with LL^T = U
U_mat = invariant_symmetric_sesquilinear_matrix(q,la)[0]
L = U_mat.cholesky();
rho_tilde = L.transpose()*rho(G[8])*L.transpose().inverse()
print("orthogonal factorization =", U_mat == L*L.transpose())
print("check orthogonality =", rho_tilde.transpose()*rho_tilde == identity_matrix(d_rho))

orthogonal factorization = True
check orthogonality = True


In [37]:
"""
we use the matrix factorization U_1 = AA^* where A is not necessarily lower triangular
U_1 is the S_n-invariant symmetric bilinear form associated to the rep'n la = [3, 1], q = 3 over GF(q^2)
GAP version of the base change matrix b1 associated to u1
b1 := [[Z(3)^0, 0*Z(3), 0*Z(3)],[Z(3^2)^2,Z(3)^0, Z(3^2)], [Z(3),Z(3^2)^3,Z(3^2)^2]]
"""
F = GF(3^2)
z = F.gen()

U_1 = matrix(F,[[1,2,2],[2,1,2],[2,2,1]])

# Define the matrix using the elements from GAP
# n.b. Z(3) is the multiplicative generator of Z/3Z which is 2 == -1
b1 = matrix(F, [
    [1, 0, 0],           # First row: Z(3)^0, 0*Z(3), 0*Z(3)
    [z^2, 1, z],         # Second row: Z(3^2)^2, Z(3)^0, Z(3^2)
    [2, z^3, z^2]        # Third row: Z(3), Z(3^2)^3, Z(3^2)^2
])

#A = b1^{-1}
b1_star = conjugate_pos_char(b1).transpose()
U_1 == b1.inverse()*b1_star.inverse()

True